In [118]:
import sys
import torch
sys.path.append("..")
REF_PATH = "/home/yifan/dataset/noisy/pairflip/cifar10/noisy0.001"
CLEAN_PATH = "/home/yifan/dataset/confusion/pairflip/cifar10/0"



sys.path.append(REF_PATH)
sys.path.append(CLEAN_PATH)

from config import config
CLASSES = config["CLASSES"]
GPU_ID = config["GPU"]
TRAINING_PARAMETER = config["TRAINING"]


NET = TRAINING_PARAMETER["NET"]

DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

import Model.model as subject_model
model = eval("subject_model.{}()".format(NET))
clean_model = eval("subject_model.{}()".format(NET))


model_location = os.path.join(REF_PATH,'Model', "Epoch_{:d}".format(200), "subject_model.pth")

model.load_state_dict(torch.load(model_location, map_location=torch.device("cpu")))

clean_model_location = os.path.join(CLEAN_PATH, 'Model',  "Epoch_{:d}".format(200), "subject_model.pth")
clean_model.load_state_dict(torch.load(clean_model_location, map_location=torch.device("cpu")))


<All keys matched successfully>

In [119]:
from singleVis.data import NormalDataProvider
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]
ref_provider = NormalDataProvider(REF_PATH, model, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
data_provider = NormalDataProvider(CLEAN_PATH, model, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)

print("ref_provider",ref_provider.prediction_function(200))


Finish initialization...
Finish initialization...
ref_provider Sequential(
  (0): Linear(in_features=512, out_features=10, bias=True)
)


In [165]:
i=0
import numpy as np

import json
with open ('/home/yifan/projects/deepdebugertool/DLVisDebugger/reptransformationHelper/confusion.json', "r") as f:
	R = json.load(f)
import numpy as np
from sklearn.cross_decomposition import CCA
import torch.nn.functional as F
# for param_q, param_k in zip(model.parameters(), clean_model.parameters()):
# 	print("i")
# imgs = null
# a = 1
# for name, module in model._modules.items():
# 	imgs = module(a)
# 	print("name",name,imgs)

# for name, module in model._modules['body']._modules.items():
# 	print("name",name,imgs)
# print(model.fc.out_features)
# list(model.parameters()) + list(clean_model.parameters())
Xi_pred = ref_provider.get_pred(200, ref_provider.train_representation(200))
Xj_pred = data_provider.get_pred(200, data_provider.train_representation(200))

xjR__pred = ref_provider.get_pred(200, np.dot(data_provider.train_representation(200),R))

xjjR__pred = data_provider.get_pred(200, np.dot(data_provider.train_representation(200),R))

def regression_loss(x, y):
    x = F.normalize(x, dim=1)
    y = F.normalize(y, dim=1)
    return 2 - 2 * (x * y).sum(dim=-1)
	
print(regression_loss(torch.tensor(Xi_pred),torch.tensor(Xj_pred)).mean())
print(regression_loss(torch.tensor(Xi_pred),torch.tensor(xjR__pred)).mean())

print(regression_loss(torch.tensor(Xj_pred),torch.tensor(xjjR__pred)).mean())

print(regression_loss(torch.tensor(Xj_pred),torch.tensor(xjR__pred)).mean())
# print(regression_loss(torch.tensor(ref_pred),torch.tensor(xj_pred)).mean())
# print(regression_loss(torch.tensor(con_pred),torch.tensor(xj_pred)).mean())
# print(regression_loss(torch.tensor(con_pred),torch.tensor(xj_pred)).mean())
loss2 = F.mse_loss(torch.tensor(Xj_pred),torch.tensor(xjR__pred))
print("loss2",loss2)
# print("loss:" F.mse_loss(torch.tensor(ref_pred),torch.tensor(con_pred)))


100%|██████████| 250/250 [00:00<00:00, 8762.89it/s]

tensor(0.4228)
tensor(0.2349)
tensor(1.5428)
tensor(0.3373)
loss2 tensor(32.9107)


In [158]:
import numpy as np
from sklearn.cross_decomposition import CCA

# 建立模型
X = Xj_pred
Y = xjR__pred
cca = CCA(n_components=1)
# 训练数据
cca.fit(X, Y)
# print(X)
X_train_r, Y_train_r = cca.transform(X, Y)
# print(X_train_r)
print('Xj vs XjR ')
print(np.corrcoef(X_train_r[:, 0], Y_train_r[:, 0])[0, 1]) #输出相关系数  

Xj vs XjR 
0.9767734778018012
